# Proyecto: Clasificacion Supervisada de Riesgo de Inundacion por Parroquia (Guayas)

## Objetivo
Disenar e implementar un modelo supervisado que clasifique el riesgo de inundacion por parroquia,
cumpliendo las restricciones del proyecto:

- uso de datos reales de fuentes oficiales del Ecuador,
- sin etiquetas de riesgo predefinidas por terceros,
- comparacion de Regresion Logistica, Arbol de Decision y Ensamble,
- optimizacion con GridSearchCV,
- evaluacion con Precision, Recall, F1 y ROC-AUC.

## Alcance de este notebook
Este cuaderno esta alineado con el pipeline del repositorio (`ml/train_and_prepare.py`) y documenta:

1. limpieza y control de calidad,
2. construccion de etiqueta objetivo supervisada,
3. ingenieria de variables (incluye variable derivada/transformada),
4. entrenamiento y optimizacion,
5. validacion de cobertura total de parroquias de Guayas,
6. coherencia con artefactos de la aplicacion web.


## 1) Preparacion del entorno

Este notebook funciona en local o Colab si el repositorio esta clonado.


In [ ]:
from __future__ import annotations

import json
import subprocess
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    RocCurveDisplay,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier


def find_project_root() -> Path:
    candidates = [
        Path.cwd(),
        Path.cwd().parent,
        Path('/content/entregables'),
        Path('/content'),
    ]
    for candidate in candidates:
        if (candidate / 'ml' / 'train_and_prepare.py').exists():
            return candidate.resolve()
    raise FileNotFoundError('No se encontro la raiz del proyecto con ml/train_and_prepare.py')


ROOT = find_project_root()
DATA_RAW = ROOT / 'data' / 'raw'
OUTPUTS = ROOT / 'outputs'
APP_DATA = ROOT / 'app' / 'data'

print('ROOT:', ROOT)


## 2) Fuentes oficiales y trazabilidad

Las fuentes oficiales y el criterio de etiqueta se consolidan en el archivo:

- `outputs/fuentes_y_metodologia_oficial.json`

Si aun no existe, se generara al ejecutar el pipeline oficial.


In [ ]:
source_registry_path = OUTPUTS / 'fuentes_y_metodologia_oficial.json'

if source_registry_path.exists():
    source_registry = json.loads(source_registry_path.read_text(encoding='utf-8'))
    source_registry
else:
    print('Aun no existe fuentes_y_metodologia_oficial.json. Se generara en la seccion de pipeline.')


## 3) Carga y limpieza del dataset historico base

Se usa `data/raw/dataset_proyecto.csv` para construir la etiqueta supervisada a partir de eventos historicos.

Controles aplicados:

- normalizacion de nombres de columna,
- tipado numerico,
- parseo de fecha (`anio`, `mes`),
- eliminacion de duplicados,
- reporte de faltantes,
- chequeo de balance de la variable `inundacion`.


In [ ]:
historical_path = DATA_RAW / 'dataset_proyecto.csv'
if not historical_path.exists():
    raise FileNotFoundError(f'No existe {historical_path}')

raw_df = pd.read_csv(historical_path)

rename_map = {
    'Codigo': 'codigo',
    'Código': 'codigo',
    'Nombre de provincia': 'provincia',
    'Nombre de canton': 'canton',
    'Nombre de parroquia': 'parroquia',
    'anio': 'anio',
    'mes': 'mes',
    'inundacion': 'inundacion',
}

df_hist = raw_df.rename(columns=rename_map).copy()

for col in ['codigo', 'anio', 'mes', 'inundacion']:
    if col in df_hist.columns:
        df_hist[col] = pd.to_numeric(df_hist[col], errors='coerce')

before = len(df_hist)
df_hist = df_hist.drop_duplicates().copy()
after = len(df_hist)

if {'anio', 'mes'}.issubset(df_hist.columns):
    df_hist['fecha'] = pd.to_datetime(
        dict(
            year=df_hist['anio'].astype('Int64'),
            month=df_hist['mes'].astype('Int64'),
            day=1,
        ),
        errors='coerce',
    )

print(f'Registros antes de duplicados: {before}')
print(f'Registros despues de duplicados: {after}')
print()
print('Faltantes (top 12):')
print(df_hist.isna().sum().sort_values(ascending=False).head(12))


In [ ]:
if 'inundacion' not in df_hist.columns:
    raise ValueError('No existe la columna inundacion en dataset_proyecto.csv')

class_count = df_hist['inundacion'].value_counts(dropna=False).sort_index()
class_share = df_hist['inundacion'].value_counts(normalize=True, dropna=False).sort_index()

print('Conteo de clases:')
print(class_count)
print()
print('Proporcion de clases:')
print(class_share)

ax = class_count.plot(kind='bar', color=['#2ea66f', '#df4f43'], title='Balance de inundacion historica')
ax.set_xlabel('Clase inundacion')
ax.set_ylabel('Frecuencia')
plt.show()


## 4) Construccion de la variable objetivo supervisada

Se construye una etiqueta parroquial `target_alto_riesgo` desde la tasa historica de inundacion:

- `tasa_inundacion_historica = eventos_inundacion / total_periodos`
- umbral tecnico: percentil 66 de la tasa historica.

Esto cumple la restriccion de no usar etiquetas predefinidas de riesgo.


In [ ]:
def normalize_code(value: object) -> str:
    digits = ''.join(ch for ch in str(value) if ch.isdigit())
    digits = digits.lstrip('0')
    return digits


def safe_div(num: pd.Series, den: pd.Series) -> pd.Series:
    den_safe = den.replace({0: np.nan})
    out = num / den_safe
    return out.replace([np.inf, -np.inf], np.nan)


if 'codigo' not in df_hist.columns:
    raise ValueError('No existe codigo/Codigo en dataset_proyecto.csv')

hist_label = df_hist.copy()
hist_label['codigo'] = hist_label['codigo'].map(normalize_code)
hist_label['inundacion'] = pd.to_numeric(hist_label['inundacion'], errors='coerce').fillna(0).astype(int)

labels_df = hist_label.groupby('codigo', as_index=False).agg(
    eventos_inundacion=('inundacion', 'sum'),
    total_periodos=('inundacion', 'count'),
)
labels_df['tasa_inundacion_historica'] = safe_div(
    labels_df['eventos_inundacion'], labels_df['total_periodos']
).fillna(0)

threshold = float(labels_df['tasa_inundacion_historica'].quantile(0.66))
labels_df['target_alto_riesgo'] = (
    labels_df['tasa_inundacion_historica'] >= threshold
).astype(int)

print('Parroquias con historial:', len(labels_df))
print('Umbral (percentil 66):', round(threshold, 6))
labels_df.head()


## 5) Ejecucion del pipeline oficial del repositorio

Se ejecuta `ml/train_and_prepare.py` para:

- descargar datos oficiales faltantes (INEC MANLOC),
- integrar geometria parroquial oficial,
- entrenar modelos (RL, DT, ensamble, DT optimizado),
- generar predicciones para todas las parroquias de Guayas,
- exportar archivos para la app Flask.


In [ ]:
script_path = ROOT / 'ml' / 'train_and_prepare.py'
if not script_path.exists():
    raise FileNotFoundError(script_path)

print('Ejecutando:', script_path)
subprocess.run([sys.executable, str(script_path)], check=True)


## 6) Verificacion de cobertura completa en Guayas

Se valida que la salida final cubra todas las parroquias oficiales de Guayas y sin faltantes de riesgo/probabilidad.


In [ ]:
pred_path = OUTPUTS / 'predicciones_parroquias.csv'
full_dataset_path = OUTPUTS / 'dataset_guayas_oficial_completo.csv'
missing_pred_path = OUTPUTS / 'parroquias_guayas_sin_historial_predichas.csv'

pred_df = pd.read_csv(pred_path, dtype={'codigo': str})
full_df = pd.read_csv(full_dataset_path, dtype={'codigo': str})
missing_pred_df = pd.read_csv(missing_pred_path, dtype={'codigo': str})

pred_df['codigo'] = pred_df['codigo'].str.zfill(6)
full_df['codigo'] = full_df['codigo'].str.zfill(6)

print('Parroquias oficiales de Guayas:', full_df['codigo'].nunique())
print('Parroquias predichas:', pred_df['codigo'].nunique())
print('Probabilidades faltantes:', int(pred_df['probabilidad_inundacion'].isna().sum()))
print('Riesgos faltantes:', int(pred_df['riesgo_categoria'].isna().sum()))
print('Parroquias sin historial (predichas):', len(missing_pred_df))

pred_df[['codigo', 'provincia', 'canton', 'parroquia', 'probabilidad_inundacion', 'riesgo_categoria']].head()


## 7) Ingenieria de variables para modelado

Variables usadas (coherentes con `ml/train_and_prepare.py`):

- superficie y forma: `superficie_km2`, `shape_length`, `indice_compacidad`
- localizacion: `latitud`, `longitud`
- demografia oficial INEC 2022: `poblacion_2022`, `hogares_2022`, `viviendas_2022`
- composicion: `edad_promedio_2022`, `pct_mujeres_2022`, `pct_urbana_2022`
- derivadas: `densidad_poblacional_2022`, `personas_por_hogar_2022`, `viviendas_por_km2_2022`

Adicionalmente se muestra una transformacion derivada en notebook:

- `log_densidad_poblacional_2022 = log1p(densidad_poblacional_2022)`


In [ ]:
model_df = full_df.copy()

model_df['log_densidad_poblacional_2022'] = np.log1p(
    pd.to_numeric(model_df['densidad_poblacional_2022'], errors='coerce').clip(lower=0)
)

feature_cols = [
    'superficie_km2',
    'shape_length',
    'latitud',
    'longitud',
    'poblacion_2022',
    'hogares_2022',
    'viviendas_2022',
    'edad_promedio_2022',
    'pct_mujeres_2022',
    'pct_urbana_2022',
    'densidad_poblacional_2022',
    'personas_por_hogar_2022',
    'viviendas_por_km2_2022',
    'indice_compacidad',
    'log_densidad_poblacional_2022',
]

labeled_df = model_df[model_df['target_alto_riesgo'].notna()].copy()
unlabeled_df = model_df[model_df['target_alto_riesgo'].isna()].copy()

X = labeled_df[feature_cols]
y = labeled_df['target_alto_riesgo'].astype(int)

print('Shape X:', X.shape)
print('Distribucion target:')
print(y.value_counts().sort_index())


## 8) Entrenamiento de modelos requeridos

Se entrena:

1. Regresion Logistica (base)
2. Arbol de Decision
3. Ensamble (RL + DT + RF)
4. Arbol optimizado con GridSearchCV (priorizando Recall)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y,
)

pre_scaled = ColumnTransformer(
    transformers=[
        (
            'num',
            Pipeline(
                steps=[
                    ('imputer', SimpleImputer(strategy='median')),
                    ('scaler', StandardScaler()),
                ]
            ),
            feature_cols,
        )
    ]
)

pre_unscaled = ColumnTransformer(
    transformers=[
        (
            'num',
            Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
            feature_cols,
        )
    ]
)

model_lr = Pipeline(
    steps=[
        ('preprocess', pre_scaled),
        ('clf', LogisticRegression(max_iter=2000, class_weight='balanced', random_state=42)),
    ]
)

model_dt = Pipeline(
    steps=[
        ('preprocess', pre_unscaled),
        ('clf', DecisionTreeClassifier(random_state=42, class_weight='balanced', min_samples_leaf=2)),
    ]
)

model_rf = Pipeline(
    steps=[
        ('preprocess', pre_unscaled),
        (
            'clf',
            RandomForestClassifier(
                n_estimators=300,
                random_state=42,
                class_weight='balanced_subsample',
                min_samples_leaf=2,
            ),
        ),
    ]
)

model_ens = VotingClassifier(
    estimators=[('lr', model_lr), ('dt', model_dt), ('rf', model_rf)],
    voting='soft',
)

model_lr.fit(X_train, y_train)
model_dt.fit(X_train, y_train)
model_ens.fit(X_train, y_train)

grid_dt = GridSearchCV(
    estimator=model_dt,
    param_grid={
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_depth': [3, 5, 8, None],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4],
    },
    scoring='recall',
    cv=5,
    n_jobs=1,
    refit=True,
)

grid_dt.fit(X_train, y_train)
model_dt_opt = grid_dt.best_estimator_

print('Mejores hiperparametros (DT):', grid_dt.best_params_)


## 9) Evaluacion comparativa de modelos

Metricas reportadas:

- Precision
- Recall (metrica prioritaria)
- F1-score
- ROC-AUC

Criterio de seleccion final del notebook:

1. mayor Recall,
2. desempate con F1,
3. desempate con ROC-AUC.


In [ ]:
def eval_model(name: str, model, x_test: pd.DataFrame, y_test: pd.Series) -> dict:
    y_pred = model.predict(x_test)
    y_score = model.predict_proba(x_test)[:, 1]
    return {
        'modelo': name,
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_test, y_score),
        'y_pred': y_pred,
        'y_score': y_score,
    }

results = [
    eval_model('Regresion Logistica (Base)', model_lr, X_test, y_test),
    eval_model('Arbol de Decision', model_dt, X_test, y_test),
    eval_model('Ensamble RL+DT+RF', model_ens, X_test, y_test),
    eval_model('Arbol de Decision Optimizado (GridSearchCV)', model_dt_opt, X_test, y_test),
]

metrics_table = pd.DataFrame([
    {
        'modelo': r['modelo'],
        'precision': round(r['precision'], 4),
        'recall': round(r['recall'], 4),
        'f1': round(r['f1'], 4),
        'roc_auc': round(r['roc_auc'], 4),
    }
    for r in results
]).sort_values(['recall', 'f1', 'roc_auc'], ascending=False)

metrics_table


In [ ]:
best_model_name = metrics_table.iloc[0]['modelo']
best_result = next(r for r in results if r['modelo'] == best_model_name)

print('Modelo seleccionado (criterio recall):', best_model_name)
print()
print('Reporte de clasificacion:')
print(classification_report(y_test, best_result['y_pred'], digits=4))


In [ ]:
plt.figure(figsize=(8, 6))
for r in results:
    RocCurveDisplay.from_predictions(y_test, r['y_score'], name=r['modelo'])

plt.title('Curvas ROC - Comparacion de modelos')
plt.grid(alpha=0.25)
plt.show()


## 10) Prediccion de parroquias sin historial y analisis de salida

Se aplica el mejor modelo para completar parroquias que no tenian historial etiquetado,
sin usar datos sinteticos.


In [ ]:
best_model = {
    'Regresion Logistica (Base)': model_lr,
    'Arbol de Decision': model_dt,
    'Ensamble RL+DT+RF': model_ens,
    'Arbol de Decision Optimizado (GridSearchCV)': model_dt_opt,
}[best_model_name]

all_probs = best_model.predict_proba(model_df[feature_cols])[:, 1]
model_df['probabilidad_reestimada'] = all_probs

q_low = float(model_df['probabilidad_reestimada'].quantile(0.33))
q_high = float(model_df['probabilidad_reestimada'].quantile(0.66))


def classify_risk(prob: float, low_threshold: float, high_threshold: float) -> str:
    if prob >= high_threshold:
        return 'Alto'
    if prob >= low_threshold:
        return 'Medio'
    return 'Bajo'


model_df['riesgo_reestimado'] = model_df['probabilidad_reestimada'].apply(
    lambda p: classify_risk(float(p), q_low, q_high)
)

print('Distribucion de riesgo reestimado:')
print(model_df['riesgo_reestimado'].value_counts())
print()
print('Parroquias originalmente sin historial:', len(unlabeled_df))

model_df.loc[
    model_df['target_alto_riesgo'].isna(),
    ['codigo', 'parroquia', 'probabilidad_reestimada', 'riesgo_reestimado']
].head(10)


## 11) Coherencia con la aplicacion web (Flask + Leaflet)

Se verifica que los artefactos consumidos por la web esten sincronizados:

- `app/data/predicciones_parroquias.csv`
- `app/data/parroquias_riesgo.geojson`


In [ ]:
app_pred_path = APP_DATA / 'predicciones_parroquias.csv'
app_geojson_path = APP_DATA / 'parroquias_riesgo.geojson'

app_pred_df = pd.read_csv(app_pred_path, dtype={'codigo': str})
geo = json.loads(app_geojson_path.read_text(encoding='utf-8'))

guayas_features = [
    f for f in geo.get('features', [])
    if str(f.get('properties', {}).get('provincia', '')).strip().upper() == 'GUAYAS'
]

missing_geo_risk = sum(1 for f in guayas_features if not f.get('properties', {}).get('riesgo'))
missing_geo_prob = sum(
    1 for f in guayas_features
    if f.get('properties', {}).get('probabilidad') in (None, '', 'NaN')
)

print('CSV app - filas:', len(app_pred_df))
print('GeoJSON - features Guayas:', len(guayas_features))
print('GeoJSON faltantes riesgo:', missing_geo_risk)
print('GeoJSON faltantes probabilidad:', missing_geo_prob)


## 12) Conclusiones tecnicas

- Se cumple el flujo supervisado exigido: RL, DT, ensamble y DT optimizado con GridSearchCV.
- La etiqueta objetivo fue construida tecnicamente desde historial de inundacion (sin etiquetas predefinidas).
- Se prioriza Recall por contexto de gestion de riesgo (evitar falsos negativos).
- La cobertura final incluye todas las parroquias oficiales de Guayas y completa probabilidades/riesgos faltantes.
- La salida queda integrada al mapa web productivo mediante CSV + GeoJSON.
